In [ ]:
import telebot #Biblioteca de criação do Bot Telegram
from time import sleep 
from datetime import date
from dateutil.relativedelta import relativedelta
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton #Biblioteca dos botões e markup

meses = []
nome_meses = ['JANEIRO', 'FEVEREIRO', 'MARÇO', 'ABRIL', 'MAIO', 'JUNHO', 'JULHO', 'AGOSTO', 'SETEMBRO', 'OUTUBRO', 'NOVEMBRO', 'DEZEMBRO']
now = date.today()

for i in range(3,-1,-1):
    x = relativedelta(months=i)
    meses.append({nome_meses[((now - x).month)-1]:(now - x).month})
    
def junta_dici(ld): #Função 
    resultado = {}
    for dici in ld:
        resultado.update(dici)
    return resultado
    
    
meses_junto = junta_dici(meses)
lis_dici = [{'ID':'0','lj1':False,'lj2':False,'lj3':False,'lj4':False,'lj5':False,'lj6':False,'lj7':False,'pdf':False, 'mes':now.month}]

def insere_ld(id_,item, mes=0):
    global lis_dici
    for i in lis_dici:
        if i['ID'] == id_:
            if not mes:
                i[item] = True
                return True
            else:
                i['mes'] = item
                return True
    lis_dici.append({'ID':id_,'lj1':False,'lj2':False,'lj3':False,'lj4':False,'lj5':False,'lj6':False,'lj7':False,'pdf':False, 'mes':now.month})
    insere_ld(id_, item, mes)

    
def limpa_ld(id_):
    global lis_dici
    for i in lis_dici:
        if i['ID'] == id_:
            for k,v in i.items():
                if k != 'ID' or k != 'mes':
                    i[k] = False
    

def pega_escolhas(id_):
    global lis_dici
    for i in lis_dici:
        if i['ID'] == id_:
            return i
    
    
def uma_linha(nome_arquivo):
    with open (f'C:/Users/pdv/Desktop/gabriel/{nome_arquivo}', 'r') as arquivo:
        for linha in arquivo:
            return linha        

            
chaveapi = uma_linha('novachave.txt')      
bot = telebot.TeleBot(chaveapi) #Cria uma instância do bot

def faz_botao(dicionario): #Função pra criar os botões de resposta
    markup = InlineKeyboardMarkup() #inicia o markup (marcação)
    for botao, retorno in dicionario.items(): #Desempacota os botões e seus retornos do dicionário
        markup.add(InlineKeyboardButton(str(botao), callback_data=str(retorno))) #cria o botão dentro do markup
    return markup #retorna o markup
    
    
@bot.message_handler(commands=["notas"]) #Decorador que cuida do comando /notas
def notas(mensagem):
    global meses_junto
    id_ = mensagem.chat.id #O id é uma parte da estrutura aninhada da mensagem ({'content_type': 'text', 'id': 9, 'message_id': 9,'from_user': {'id': 945441763, 'is_bot': False, 'first_name': 'Gbr', 'username': 'Gbrdoidao', 'last_name': 'Doidao', 'language_code': 'pt-br', 'can_join_groups': None, 'can_read_all_group_messages': None, 'supports_inline_queries': None}, 'date': 1649940793,'chat': {'id': 945441763, 'type': 'private', 'title': None, 'username': 'Gbrdoidao', 'first_name': 'Gbr', 'last_name': 'Doidao', 'photo': None, 'bio': None, 'description': None, 'invite_link': None, 'pinned_message': None, 'permissions': None, 'slow_mode_delay': None, 'message_auto_delete_time': None, 'sticker_set_name': None, 'can_set_sticker_set': None, 'linked_chat_id': None, 'location': None},'sender_chat': None, 'forward_from': None, 'forward_from_chat': None, 'forward_from_message_id': None, 'forward_signature': None, 'forward_sender_name': None, 'forward_date': None, 'reply_to_message': None, 'via_bot': None, 'edit_date': None, 'media_group_id': None, 'author_signature': None, 'text': 'ao', 'entities': None, 'caption_entities': None, 'audio': None, 'document': None, 'photo': None, 'sticker': None, 'video': None, 'video_note': None, 'voice': None, 'caption': None, 'contact': None, 'location': None, 'venue': None, 'animation': None, 'dice': None, 'new_chat_member': None, 'new_chat_members': None, 'left_chat_member': None, 'new_chat_title': None, 'new_chat_photo': None, 'delete_chat_photo': None, 'group_chat_created': None, 'supergroup_chat_created': None, 'channel_chat_created': None, 'migrate_to_chat_id': None, 'migrate_from_chat_id': None, 'pinned_message': None, 'invoice': None, 'successful_payment': None, 'connected_website': None, 'reply_markup': None, 'json': {'message_id': 9, 'from': {'id': 945441763, 'is_bot': False, 'first_name': 'Gbr', 'last_name': 'Doidao', 'username': 'Gbrdoidao', 'language_code': 'pt-br'}, 'chat': {'id': 945441763, 'first_name': 'Gbr', 'last_name': 'Doidao', 'username': 'Gbrdoidao', 'type': 'private'}, 'date': 1649940793, 'text': 'ao'}})
    fornecedor = mensagem.text[7:] #Pega a parte importante da mensagem (sem o /notas )
    print(mensagem.from_user.username,' pesquisou notas do ', fornecedor)
    dicio_pdf = {'Preço Notas':'preconf','Nota em PDF':'pdf'} #Botão pdf / preço {'nome_botão':'retorno do botão'}
    dicio_lojas = {'LJ1 - Guanabara' : 'lj1' ,'LJ2 - Primavera' : 'lj2','LJ3 - Iguatemi' : 'lj3','LJ4 - Flamboyant' : 'lj4', 'LJ5 - São Paulo' : 'lj5','LJ6 - Valinhos' : 'lj6',
                   'LJ7 - Ouro Verde' : 'lj7', 'ESCOLHIDO' : 'escolheu'} #Botões das lojas e seus retornos 
    bot.send_message(id_, ' Qual Mes? ', reply_markup=faz_botao(meses_junto)) #Botao pro user escolher o mes da nota
    bot.send_message(id_, ' Qual vc prefere? ' ,reply_markup=faz_botao(dicio_pdf)) #Cria o botão pro escolher entre PDF e meu preço
    bot.send_message(id_, ' LOJAS: ' ,reply_markup=faz_botao(dicio_lojas)) #Cria o botão pro escolher entre PDF e meu preço
    bot.reply_to(mensagem, 'pronto ;)') #Mensagem pra mostrar que deu certo
    

@bot.message_handler(func=lambda message: True) #Decorador que cuida de mensagens (não comandos)
def resposta(mensagem):
    texto = '''Olá! Seja bem vindo ao Fartura Bot  
    Estes são os meus Comandos:
    =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=''' #Mensagem padrão de resposta
    bot.reply_to(mensagem, texto) #Responde a mensagem dele com o texto padrão
    
    
@bot.callback_query_handler(func=lambda call: True)
def handle_query(chamado):
    nome = chamado.message.from_user.username
    id_ = chamado.message.chat.id
    retorno = chamado.data
    inteiro = False
    try:
        inteiro = int(retorno)
    except:
        pass
    if retorno.startswith('lj') or retorno == 'pdf':
        insere_ld(id_,retorno)
    elif inteiro and inteiro in range(13): #Pega o retorno dos botões de meses
        insere_ld(id_, retorno, mes=1)
    elif retorno == ('escolheu'):
        escolhas = pega_escolhas(id_)
        limpa_ld(id_)
    
    
while True:
    try:
        bot.polling(none_stop=True, interval=0, timeout=0) #Mantém o bot rodando infinitamente
    except:
        sleep(10) #se der erro ele espera 10 segundos


In [26]:
import time

def calcula_tempo(funcao):
    def wrapper(*args, **kwargs):     
        tempo_comeco = time.time()
        x = funcao(*args, **kwargs)
        tempo_final = time.time()
        print(f'sua função demorou {tempo_final - tempo_comeco}')
        return x
    return wrapper



        
        
l1 = list(range(-200,4000))
l2 = list(range(3000))
l3 = []

@calcula_tempo
def checa_lista(l1,l2,l3):
    for i in l1:
        if i not in l2:
            l3.append(i)
    return l3

print(checa_lista(l1,l2,l3))

@calcula_tempo
def Diff(l1, l2):
    return list(set(l1) - set(l2)) + list(set(l2) - set(l1))

print(Diff(l1,l2))

sua função demorou 0.08380389213562012
[-200, -199, -198, -197, -196, -195, -194, -193, -192, -191, -190, -189, -188, -187, -186, -185, -184, -183, -182, -181, -180, -179, -178, -177, -176, -175, -174, -173, -172, -171, -170, -169, -168, -167, -166, -165, -164, -163, -162, -161, -160, -159, -158, -157, -156, -155, -154, -153, -152, -151, -150, -149, -148, -147, -146, -145, -144, -143, -142, -141, -140, -139, -138, -137, -136, -135, -134, -133, -132, -131, -130, -129, -128, -127, -126, -125, -124, -123, -122, -121, -120, -119, -118, -117, -116, -115, -114, -113, -112, -111, -110, -109, -108, -107, -106, -105, -104, -103, -102, -101, -100, -99, -98, -97, -96, -95, -94, -93, -92, -91, -90, -89, -88, -87, -86, -85, -84, -83, -82, -81, -80, -79, -78, -77, -76, -75, -74, -73, -72, -71, -70, -69, -68, -67, -66, -65, -64, -63, -62, -61, -60, -59, -58, -57, -56, -55, -54, -53, -52, -51, -50, -49, -48, -47, -46, -45, -44, -43, -42, -41, -40, -39, -38, -37, -36, -35, -34, -33, -32, -31, -30, -29,

In [27]:
print(0.08380389213562012 / 0.0010008811950683594)

83.73010957598856
